Example of the process of doing DEMs for all orbits for a given observation of an AR – where we don't window out any intervals because of shifts, etc. Instead, we will save energy-specific information about the percent of NuSTAR emission in the chosen region as a function of time, for later inspection. This will help identify cases where pointing shifts might cause unphysical distortion to the DEM inputs. 

Note: auto-download of an AIA file (for use in the initial co-alignmnet) will break while the JSOC is still down. We will add a thing where you can point to an existing AIA file instead.

Overview:

- Define orbits
- Run time interval selection
- Examine resulting intervals
- Manually establish a co-alignment shift between NuSTAR and AIA
- Automatically find co-alignment shifts + make regions for all other time intervals (note: this relies on the assumption that the COM is a good representation of the location of the brightest source, i.e. that the NuSTAR data is primarially one blob).
- Save AIA region files for NCCS input
- NOT IN THIS NOTEBOOK: YOU THEN TAKE THOSE AND MAKE AIA INPUTS ON THE NCCS
- Conduct AIA/NuSTAR DEMs as a function of time, given all the above
- Plot results.
- Print some stats about "left out" times.

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import glob
# from astropy.io import fits
# from astropy import units as u
# import importlib
# import pathlib

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import nustar_dem_prep as nu
# import initial_analysis as ia
# import orbit_auto as oa
import time_interval_selection as tis
import nustar_utilities as nuutil
# import gauss2D as g2d
import all_nu_analysis as ana
import visualize_dem_results as viz
import images_and_coalignment as iac


import pickle
import pathlib
import importlib
import numpy as np
import subprocess


In [ ]:

def make_dem_scripts(keys, runname='', where='./scripts/',
                    high_temp_analysis=False):

    if high_temp_analysis:
        templatefile = where+'/dems_hitemp_key.py'
    else:
        templatefile = where+'/dems_key.py'
    
    pystrings = []
    for key in keys:
        pyfile = where+'dems_'+key+'.py'
        pystring = 'python '+pyfile+' > '+' '+where+'dems_out_'+key+'.txt &'
        #print(pystring)
        pystrings.append(pystring)
        
        with open(templatefile, 'r') as f:
            lines = f.read()
            llist = lines.split('\n')
            #print(productslist)
            llist[9] = 'key = "'+key+'"'
            #print(llist)
            newlist = '\n'.join(llist)
        
            with open(pyfile, 'w') as file_out:
                file_out.seek(0)
                file_out.write(newlist)
                file_out.truncate()


    with open(where+'run_dems.sh', 'r') as f:
        lines = f.read()
        llist = lines.split('\n')
        llist.extend(['','',])
        llist.extend(pystrings)
        llist.extend(['wait', '', 'echo "all orbit scripts finished"'])
        pylist = '\n'.join(llist)

        with open(where+runname+'_run_dems.sh', 'w') as file_out:
            file_out.seek(0)
            file_out.write(pylist)
            file_out.truncate()



with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

keys = data.keys()
print(keys)

xrt_rerun_keys = ['19-feb-16', '22-apr-16_2',
                  '11-sep-17', '12-sep-17', '13-sep-17',
                  '29-may-18_1',
                  '12-apr-19', '13-apr-19',
                  '29-jan-20', 
                  '29-apr-21',  '03-may-21_1', '03-may-21_2', 
                  '30-jul-21_1', '30-jul-21_2']

keys = xrt_rerun_keys


nan_rerun_keys = ['01-sep-15',
                  '11-sep-17', '13-sep-17',
                  '29-may-18_1',
                 '29-apr-21', '30-jul-21_1']

#keys = ['08-jun-20', '08-jan-21', '26-jul-16_2']

#reruns = ['26-jul-16_1', '19-feb-16', '02-sep-15']

make_dem_scripts(keys, runname='run_aiaxrt')

In [ ]:
#rerun all, (hightemp analysis to do rescale)

importlib.reload(ana)

with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

keys = data.keys()

thekey=['12-sep-17']
the_need_images_keys=['19-feb-16', '29-jan-20'] #run whenever (split between this and xrt_debug nb)
the_need_rerun_keys=['11-sep-17', '12-sep-17', '13-sep-17', '12-apr-19', '13-apr-19'] #run while on MSFC VPN - need IDL for responses

#for key in list(keys)[8:]:
for key in thekey:
    print(key)
    if key =='22-apr-16_1':
        continue
    ARDict = data[key]
    id_dirs = ARDict['datapaths']
    working_dir = ARDict['working_dir']
    method = ARDict['method']

    ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

    data[key] = ARDict

    with open('all_targets.pickle', 'wb') as f:
             # Pickle the 'data' dictionary using the highest protocol available.
             pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

    for id in id_dirs:
        evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
        time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
        timerange = [time0, time1]
        print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))
    
    ana.do_key_dem(key, plot_xrt=True, method=method, high_temp_analysis=False, rscl=True) #missing_last=True, missing_orbit=1, plot_xrt=False)
    print('')
    print('')


    


In [ ]:
#file = '/Users/jmdunca2/do-dem/initial_dem_7may21/all_aia_dicts_07-may-21_post/orbit_20617005001/20-59-50_22-00-03_aia_prep.pickle'
file = '/Users/jmdunca2/do-dem/initial_dem_26jul16_1/all_aia_dicts_26-jul-16_1_post/orbit_20201001001/19-22-55_20-23-20_aia_prep.pickle'



with open(file, 'rb') as f:
    data = pickle.load(f)

data.keys()
print(data['region1']['aia_dn_s_px'][-3:], data['region1']['newerr'][-3:])
print(data['region0']['aia_dn_s_px'][-3:], data['region0']['newerr'][-3:])


#file = '/Users/jmdunca2/do-dem/initial_dem_7may21/all_aia_dicts_07-may-21_post/orbit_20617005001/20-59-50_22-00-03_aia_prep.pickle'
file = '/Users/jmdunca2/do-dem/initial_dem_26jul16_1/all_aia_dicts_26-jul-16_1_post/orbit_20201001001/19-22-55_19-23-25_aia_prep.pickle'



with open(file, 'rb') as f:
    data2 = pickle.load(f)

data2.keys()
print(data2['region1']['aia_dn_s_px'][-3:], data2['region1']['newerr'][-3:])
print(data2['region0']['aia_dn_s_px'][-3:], data2['region0']['newerr'][-3:])




In [ ]:
from matplotlib import pyplot as plt

colors = ['red','orange','springgreen', 'green', 'dodgerblue', 'purple']

chanax = data['region1']['chans']
dn_in = data['region1']['aia_dn_s_px']
trmatrix = data['region1']['aia_tr']
temps = data['region1']['tresp_logt']

for i in np.arange(len(chanax)):
    plt.semilogy(temps, dn_in[i]/trmatrix[i,:],label=chanax[i],color=colors[i]) #,lw=4)

chanax = data2['region1']['chans']
dn_in = data2['region1']['aia_dn_s_px']
trmatrix = data2['region1']['aia_tr']
temps = data2['region1']['tresp_logt']

for i in np.arange(len(chanax)):
    plt.semilogy(temps, dn_in[i]/trmatrix[i,:],label=chanax[i], linestyle=':',color=colors[i])#,color=clrs[i],lw=4)

plt.xlim(5.7,7.2)
plt.ylim(1e26,1e33)


all_loci = np.zeros((len(chanax), len(temps)))
for i in np.arange(len(chanax)):
    all_loci[i,:] =  dn_in[i]/trmatrix[i,:]


min_loci = np.min(all_loci, axis=0)


plt.semilogy(temps, min_loci, label='minima', color='black')

plt.legend()

In [ ]:
#get minimum loci value as a function of temperature

all_loci = np.zeros((len(chanax), len(temps)))
for i in np.arange(len(chanax)):
    all_loci[i,:] =  dn_in[i]/trmatrix[i,:]


print(all_loci)

min_loci = np.min(all_loci, axis=0)
min_loci


In [ ]:
file = '/Users/jmdunca2/do-dem/initial_dem_29apr21/14-53-45_14-59-45/14-53-45_14-59-45_5.6_7.2_29-apr-21_MC_DEM_result.pickle'

with open(file, 'rb') as f:
    data = pickle.load(f)

data.keys()

In [ ]:
# importlib.reload(ana)
# key = '26-jul-16_2'

# #===============================================================
# #Update dictionary to have correct aia path for NCCS-prepped data.
# #===============================================================
# with open('all_targets.pickle', 'rb') as f:
#     data = pickle.load(f)

# ARDict = data[key]

# id_dirs = ARDict['datapaths']
# obsids = ARDict['obsids']
# working_dir = ARDict['working_dir']
# method = ARDict['method']

# ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

# data[key] = ARDict

# with open('all_targets.pickle', 'wb') as f:
#          # Pickle the 'data' dictionary using the highest protocol available.
#          pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

# #===============================================================
# #DO DEMS


# for id in id_dirs:
#     evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
#     time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
#     timerange = [time0, time1]
#     print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

# ana.do_key_dem(key, plot_xrt=False, method=method, high_temp_analysis=True, rscl=True) #missing_last=True, missing_orbit=1, plot_xrt=False)

# #===============================================================




In [ ]:
import glob
o=2
prepped_aia_dir=ARDict['prepped_aia']
print(prepped_aia_dir)
orbit_aia_dir = prepped_aia_dir+'/orbit_'+obsids[o]+'/'
print(orbit_aia_dir)
files = glob.glob(orbit_aia_dir+'/*')

for filename in files: 
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    #print(data)
    newdata = data['region0']

    with open(filename, 'wb') as f:
        pickle.dump(newdata, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def move_dems(key, method):


    with open('all_targets.pickle', 'rb') as f:
        data = pickle.load(f)
    
    ARDict = data[key]
    
    id_dirs = ARDict['datapaths']
    obsids = ARDict['obsids']
    working_dir = ARDict['working_dir']
    print(working_dir)

    minT=5.6
    maxT=7.2
    
    if method == 'fit':
        all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                    missing_last=False)
    
        for time in all_time_intervals_list:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
        
            demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            print(demfile)
    
            status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 
    
    
    if method in ['input', 'double']:
        if method=='input':
            region_dirs = iac.find_region_dirs(working_dir)
        if method=='double':
            gauss_stats = ARDict['gauss_stats']
            sep_axis = gauss_stats[0][0]
            region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
            #print(region_dirs)
    
        all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 
    
        for i in range(0, len(region_dirs)):
            atl = all_all_time_intervals_list[i]
            r = region_dirs[i]
    
            for time in atl:
                timestring = time[0].strftime('%H-%M-%S')
                stopstring = time[1].strftime('%H-%M-%S')
                timestring=timestring+'_'+stopstring
            
                demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
                demfile_ = demfile.split('/')[-1][0:-7]+'_region_'+str(i)+'.pickle'
                print(demfile_)
        
                status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
        
 

keys = ['01-sep-15', '02-sep-15',
        '19-feb-16', '22-apr-16_2', '26-jul-16_1', '27-jul-16_1', '26-jul-16_2',
        '11-sep-17', '12-sep-17', '13-sep-17', '10-oct-17',
        '29-may-18_1', '09-sep-18', '10-sep-18', 
        '12-apr-19', '13-apr-19', 
        '06-jun-20', '07-jun-20', 
        '08-jun-20', '09-jun-20',
        '29-apr-21', '03-may-21_1', '03-may-21_2', '20-jul-21', 
        '30-jul-21_1', 
        '30-jul-21_2']

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)

for k in keys:
    method = all_targets[k]['method']
    move_dems(k, method)



In [ ]:
demfile_

In [ ]:
# importlib.reload(ana)
# key = '30-jul-21_1'
# method='double'

# #===============================================================
# #Update dictionary to have correct aia path for NCCS-prepped data.
# #===============================================================
# with open('all_targets.pickle', 'rb') as f:
#     data = pickle.load(f)

# ARDict = data[key]

# id_dirs = ARDict['datapaths']
# obsids = ARDict['obsids']
# working_dir = ARDict['working_dir']

# ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

# data[key] = ARDict

# with open('all_targets.pickle', 'wb') as f:
#          # Pickle the 'data' dictionary using the highest protocol available.
#          pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

# #===============================================================
# #DO DEMS

# import initial_analysis as ia
# import nustar_utilities as nuutil

# for id in id_dirs:
#     evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
#     time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
#     timerange = [time0, time1]
#     print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

# ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

# #===============================================================


# #===============================================================
# #Copy DEM result files to common directory


# minT=5.6
# maxT=7.2

# if method == 'fit':
#     all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
#                                                                 missing_last=False)
    
#     for time in all_time_intervals_list:
#         timestring = time[0].strftime('%H-%M-%S')
#         stopstring = time[1].strftime('%H-%M-%S')
#         timestring=timestring+'_'+stopstring
        
#         demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
#         print(demfile)
    
#         status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


# if method in ['input', 'double']:
#     if method=='input':
#         region_dirs = iac.find_region_dirs(working_dir)
#     if method=='double':
#         gauss_stats = ARDict['gauss_stats']
#         sep_axis = gauss_stats[0][0]
#         region_dirs = iac.find_direction_dirs(working_dir, sep_axis)
#         #print(region_dirs)

#     all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True) 

#     for i in range(0, len(region_dirs)):
#         atl = all_all_time_intervals_list[i]
#         r = region_dirs[i]

#         for time in atl:
#             timestring = time[0].strftime('%H-%M-%S')
#             stopstring = time[1].strftime('%H-%M-%S')
#             timestring=timestring+'_'+stopstring
            
#             demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
#             demfile_ = demfile[70:-7]+'_region_'+str(i)+'.pickle'
#             print(demfile_)
        
#             status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
    
# #===============================================================



In [ ]:
importlib.reload(ana)
key = '12-sep-17'
method='input'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, plot_xrt=False, method=method) #missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

if method == 'fit':
    all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)
    
    for time in all_time_intervals_list:
        timestring = time[0].strftime('%H-%M-%S')
        stopstring = time[1].strftime('%H-%M-%S')
        timestring=timestring+'_'+stopstring
        
        demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
        print(demfile)
    
        status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 


if method == 'input':
    region_dirs = iac.find_region_dirs(working_dir)
    all_all_time_intervals, fixit, all_all_time_intervals_list = tis.region_time_intervals(region_dirs, id_dirs, shush=True, list_=True)

    for i in range(0, len(region_dirs)):
        atl = all_all_time_intervals_list[i]
        r = region_dirs[i]

        for time in atl:
            timestring = time[0].strftime('%H-%M-%S')
            stopstring = time[1].strftime('%H-%M-%S')
            timestring=timestring+'_'+stopstring
            
            demfile = r+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
            demfile_ = demfile[70:-7]+'_region_'+str(i)+'.pickle'
            print(demfile_)
        
            status = subprocess.call('cp '+demfile+' ./compact_results/'+demfile_, shell=True) 
    
#===============================================================



In [ ]:
len('/Users/jmdunca2/do-dem/initial_dem_12sep17/region_0/20-56-30_20-57-00/')

In [ ]:
importlib.reload(ana)
key = '09-sep-18'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import initial_analysis as ia
import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, missing_last=True, missing_orbit=1, plot_xrt=False)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

all_time_intervals, all_time_intervals_list = tis.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)

for time in all_time_intervals_list:
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    
    demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
    print(demfile)

    status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 

#===============================================================



In [ ]:
keys = ['01-sep-15', '02-sep-15',
        '26-jul-16_1', '27-jul-16_1',
        '11-sep-17', '12-sep-17', '13-sep-17',
        '29-may-18_1', '09-sep-18', '10-sep-18', 
        '12-apr-19', '13-apr-19', 
        '06-jun-20', '07-jun-20', 
        '08-jun-20', '09-jun-20',
        '29-apr-21', '03-may-21_1', '03-may-21_2', '20-jul-21', 
        '30-jul-21_1', 
        '30-jul-21_2']

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)
    
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

In [ ]:
import pickle
import numpy as np

aia_file = '/Users/jmdunca2/do-dem/initial_dem_6jun20/all_aia_dicts_06-jun-20_post/orbit_20611003001/20-59-25_20-59-55_aia_prep.pickle'
with open(aia_file, 'rb') as f:
    aia_dat = pickle.load(f)

np.isnan(aia_dat['aia_dn_s_px']).any()

In [ ]:
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

In [ ]:
import pandas as pd

df = pd.read_csv('fpmA.csv')
starts = df['flare_start'].values
stops = df['flare_end'].values

for i in range(0, len(starts)-1):
    print(starts[i], stops[i])

In [ ]:
for f in flaretimes:
    print(f[0], f[1])

In [ ]:
flaretimes=[]
nonflaretimes=[]
for time in all_time_intervals_list:
    b4 = [s < time[0] for s in starts]
    ea = [s > time[0] for s in stops]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    b4 = [s > time[0] for s in starts]
    ea = [s < time[1] for s in starts]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    nonflaretimes.append(time)

For the time interval to overlap with a flare, it could either ENVELOP a flare, start before (and end during) start during (and end during) and start + end within the flare. 

- fstart tstart tend fend - ENVELOP
- fstart tstart fend tend - STOPSIDE
  
- tstart fstart tend fend - STARTSIDE
- tstart fstart fend tend - DURING



In [ ]:
from matplotlib import pyplot as plt 

logbins = np.geomspace(np.min(all_above10s), np.max(all_above10s), 30)

fig, ax = plt.subplots(figsize=(15,4), tight_layout = {'pad': 1})

ax.hist(all_above10s, bins=logbins, color='skyblue', edgecolor='black')
ax.set_xscale('log')
ax.axvline(1.8e22, color='Red')
ax.axvline(1.5e23, color='Red')
ax.axvspan(1.8e22, 1.5e23, alpha=0.3, color='Red', label='Ishikawa (2017) 95% Interval')
ax.set_ylabel('Number of intervals')
ax.set_xlabel('EM Integrated >10 MK')
ax.legend()

In [ ]:
# import glob

# #Set path to obsid directory - initial pipeline should have been run already.
# ind=0
# datapath=id_dirs[ind]
# obsid=obsids[ind]

# evt_data, hdr = nu.return_submap(datapath=datapath, fpm='A', return_evt_hdr=True)
# time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
# timerange = [time0.tt.datetime, time1.tt.datetime]
# from datetime import timezone
# timerange = [t.replace(tzinfo=timezone.utc) for t in timerange]

# #Comment second line if you're not using this same example nustar orbit
# #Edit it to include only the desired time interval (default- all times in file) once you've run this once
# #timerange=[]
# #timerange=[datetime.datetime(2018, 5, 29, 22, 22), datetime.datetime(2018, 5, 29, 23, 20)]

# evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
# evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
# hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
# hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

# import lightcurves as lc

# importlib.reload(lc)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[2.,4.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[4.,6.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[6.,10.], 
#                               livetime_corr=False, save_dir=working_dir)

# lc.plot_nustar_lightcurves(eranges = [[2.,4.],[4.,6.],[6.,10.]],
#                            timerange=timerange, save_dir=working_dir)